In [1]:
import os
import numpy as np
import time
import sagemaker
from sagemaker.session import s3_input
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, rule_configs
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
!pip install -q --upgrade pip
!pip install -q sagemaker smdebug awscli --upgrade

In [3]:
hyperparams={    'epochs'       : 2,
                 'learning-rate': 0.01,
                 'batch-size'   : 256,
                 'weight-decay' : 2e-4,
                 'momentum'     : 0.9,
                 'optimizer'    : 'sgd'}

In [4]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()), 
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overfit())
]

In [5]:
job_name = 'tensorflow-debugger-' + time.strftime('%Y-%m-%d-%H-%M-%S-%j', time.gmtime())
output_path = 's3://sagemaker-jobs/jobs/'
model_dir = output_path + job_name

tf_estimator = TensorFlow(entry_point          = 'cifar10-training-script-sagemaker.py', 
                          source_dir           = 'code',
                          model_dir            = model_dir,
                          output_path          = output_path,
                          code_location        = output_path,
                          role                 = role,
                          train_instance_count = 1, 
                          train_instance_type  = 'local_gpu',
                          framework_version    = '1.14', 
                          py_version           = 'py3',
                          script_mode          = True,
                          hyperparameters      = hyperparams)
#                          rules                = rules)

train_path = 's3://sagemaker-jobs/cifar10-dataset/train'
val_path = 's3://sagemaker-jobs/cifar10-dataset/validation'
eval_path = 's3://sagemaker-jobs/cifar10-dataset/eval'

In [6]:
tf_estimator.fit({'training': train_path,'validation': val_path,'eval': eval_path},
                 job_name=job_name, wait=True)

Creating tmpgs6vhi8o_algo-1-728ve_1 ... 
Attaching to tmpgs6vhi8o_algo-1-728ve_12mdone
algo-1-728ve_1  | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
algo-1-728ve_1  |   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
algo-1-728ve_1  | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
algo-1-728ve_1  |   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
algo-1-728ve_1  | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpgs6vhi8o/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1